<a href="https://colab.research.google.com/github/dclark18/MSDS_Capstone/blob/master/CLUSTERING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from io import StringIO

%matplotlib inline

import numpy as np
#import modin.pandas as pd
import matplotlib.pyplot as plt

#import cufflinks as cf
#cf.go_offline()
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

%matplotlib inline
import numpy as np
import seaborn as sns; sns.set()

#from pathos.multiprocessing import ProcessingPool as Pool
import pandas.util.testing as tm
import multiprocessing as mp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [2]:
female_drive = 'https://drive.google.com/file/d/17fVhjvXH-ckfQM1kY_4CrQH2UDlszY-L/view?usp=sharing'

file_id = female_drive.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
female_clean = pd.read_csv(csv_raw)
female_clean.head()
female_clean = female_clean.iloc[0:22990]
female_clean.tail()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,FID,Id,Bear_ID,SAMPLEID,TURNANGLE,STEPLENGTH,BEARING,OBSERVED,datetime,month,CLUSTER,disagLWM,disagMIN,disagMAX,disagBEG,disagEND,disforLWM,disforMIN,disforMAX,disforBEG,disforEND,disnatLWM,disnatMIN,disnatMAX,disnatBEG,disnatEND,distrdsLWM,distrdsMIN,distrdsMAX,distrdsBEG,distrdsEND,distwatLWM,distwatMIN,distwatMAX,distwatBEG,distwatEND,natden5LWM,natden5MIN,natden5MAX,natden5BEG,natden5END,fordenLWM,fordenMIN,fordenMAX,fordenBEG,fordenEND,rddenLWM,rddenMIN,rddenMAX,rddenBEG,rddenEND,forestLWM,forestMIN,forestMAX,forestBEG,forestEND,naturalLWM,naturalMIN,naturalMAX,naturalBEG,naturalEND,NatnegLWM,NatnegMIN,NatnegMAX,NatnegBEG,NatnegEND,STRATUM,resid
22985,22986.0,4681.0,0.0,375.0,4681.0,-56.424118,234.498194,318.328368,0.0,8/30/2012 0:00,8.0,1,4.309573,0.0,30.000000,30.0,30.0,50.381412,0.0,84.852814,0.0,0.000000,50.381412,0.0,84.852814,0.0,0.000000,6380.917791,6296.427734,6450.000000,6450.0,6296.427734,132.836058,42.426407,210.000000,210.0,42.426407,0.210908,0.205875,0.214006,0.205875,0.213725,0.225380,0.04,0.48,0.48,0.48,0.019083,0.015298,0.022438,0.021928,0.015298,0.143652,0.0,1.0,1.0,1.0,0.143652,0.0,1.0,1.0,1.0,46.071839,-30.000000,84.852814,-30.0,-30.000000,2090.0,-0.640189
22986,22987.0,4682.0,0.0,375.0,4682.0,179.173599,515.164005,193.926086,0.0,8/30/2012 0:00,8.0,1,38.593209,0.0,67.082039,30.0,30.0,6.393206,0.0,42.426407,0.0,42.426407,6.393206,0.0,42.426407,0.0,42.426407,6401.631356,6343.918457,6450.069824,6450.0,6345.975098,469.104718,210.000000,720.000000,210.0,720.000000,0.156871,0.100932,0.205875,0.205875,0.100932,0.624969,0.20,0.80,0.48,0.20,0.024917,0.021928,0.026007,0.021928,0.024987,0.798870,0.0,1.0,1.0,0.0,0.798870,0.0,1.0,1.0,0.0,-31.332584,-67.082039,42.426407,-30.0,42.426407,2090.0,-0.444709
22987,22988.0,4683.0,0.0,375.0,4683.0,-14.217464,3473.900916,0.535023,0.0,8/30/2012 0:00,8.0,1,11.498600,0.0,150.000000,30.0,0.0,242.947041,0.0,600.000000,0.0,600.000000,242.947041,0.0,600.000000,0.0,600.000000,4980.114239,3507.947998,6450.000000,6450.0,3507.947998,1132.051525,0.000000,1867.966797,210.0,1867.966797,0.085732,0.021994,0.216902,0.205875,0.092264,0.147428,0.00,1.00,0.48,0.00,0.008933,0.000000,0.023457,0.021928,0.000000,0.127941,0.0,1.0,1.0,0.0,0.127941,0.0,1.0,1.0,0.0,236.653002,-90.000000,600.000000,-30.0,600.000000,2090.0,-0.017580
22988,22989.0,4684.0,0.0,375.0,4684.0,106.585696,440.587103,121.338182,0.0,8/30/2012 0:00,8.0,1,3.067130,0.0,42.426407,30.0,0.0,131.210272,0.0,212.132034,0.0,189.736664,131.210272,0.0,212.132034,0.0,189.736664,6648.012550,6450.000000,6842.368164,6450.0,6842.368164,251.639161,210.000000,284.604981,210.0,276.586334,0.190226,0.170756,0.206251,0.205875,0.170756,0.112259,0.00,0.56,0.48,0.00,0.023548,0.021928,0.024477,0.021928,0.022438,0.100989,0.0,1.0,1.0,0.0,0.100989,0.0,1.0,1.0,0.0,128.143142,-42.426407,212.132034,-30.0,189.736664,2090.0,-0.114555
22989,22990.0,4685.0,0.0,375.0,4685.0,148.551590,240.630812,163.304076,0.0,8/30/2012 0:00,8.0,1,16.491728,0.0,30.000000,30.0,0.0,21.462339,0.0,90.000000,0.0,90.000000,21.462339,0.0,90.000000,0.0,90.000000,6493.992869,6450.000000,6542.476563,6450.0,6542.476563,310.222342,210.000000,408.044128,210.0,400.249908,0.188504,0.167851,0.205875,0.205875,0.168728,0.421216,0.00,0.56,0.48,0.00,0.024075,0.021928,0.025497,0.021928,0.025497,0.549724,0.0,1.0,1.0,0.0,0.549724,0.0,1.0,1.0,0.0,4.970611,-30.000000,90.000000,-30.0,90.000000,2090.0,-0.290843


In [3]:
male_drive = 'https://drive.google.com/file/d/1lzz2ykpEs8HNPLGUay3S5xI8MVnLVNXY/view?usp=sharing'

file_id = male_drive.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
male_clean = pd.read_csv(csv_raw)
male_clean = male_clean.iloc[0:167057]
male_clean.head()
male_clean.tail()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,FID,Id,Bear_ID,SAMPLEID,TURNANGLE,STEPLENGTH,BEARING,OBSERVED,datetime,month,CLUSTER,disagLWM,disagMIN,disagMAX,disagBEG,disagEND,disforLWM,disforMIN,disforMAX,disforBEG,disforEND,disnatLWM,disnatMIN,disnatMAX,disnatBEG,disnatEND,distrdsLWM,distrdsMIN,distrdsMAX,distrdsBEG,distrdsEND,distwatLWM,distwatMIN,distwatMAX,distwatBEG,distwatEND,natden5LWM,natden5MIN,natden5MAX,natden5BEG,natden5END,fordenLWM,fordenMIN,fordenMAX,fordenBEG,fordenEND,rddenLWM,rddenMIN,rddenMAX,rddenBEG,rddenEND,forestLWM,forestMIN,forestMAX,forestBEG,forestEND,naturalLWM,naturalMIN,naturalMAX,naturalBEG,naturalEND,NatnegLWM,NatnegMIN,NatnegMAX,NatnegBEG,NatnegEND,STRATUM,resid
167052,167053.0,6771.0,0.0,831.0,6771.0,-77.339746,457.878450,166.201538,0.0,41151.5,8.0,-1,818.303440,611.882324,1050.000000,611.882324,1050.000000,128.257179,0.0,212.132034,0.0,150.000000,128.257179,0.0,212.132034,0.0,150.000000,1743.577491,1524.106323,1959.208984,1524.106323,1959.208984,43.786998,0.0,180.000000,90.0,180.000000,0.272780,0.194266,0.327465,0.327465,0.194266,0.097744,0.00,0.76,0.76,0.00,0.069511,0.045385,0.090260,0.09026,0.045385,0.104539,0.0,1.0,1.0,0.0,0.104539,0.0,1.0,1.0,0.0,124.493927,-42.426407,212.132034,-42.426407,150.000000,15187.0,-0.312966
167053,167054.0,6772.0,0.0,831.0,6772.0,33.112707,1513.832435,276.653991,0.0,41151.5,8.0,-1,317.464433,0.000000,658.634949,611.882324,0.000000,19.335120,0.0,67.082039,0.0,42.426407,17.890094,0.0,67.082039,0.0,42.426407,1294.241063,1083.743530,1524.106323,1524.106323,1083.743530,101.628529,0.0,300.000000,90.0,300.000000,0.248384,0.193308,0.327465,0.327465,0.193308,0.484176,0.08,0.96,0.76,0.28,0.049104,0.021928,0.090260,0.09026,0.024987,0.524813,0.0,1.0,1.0,0.0,0.564716,0.0,1.0,1.0,0.0,-6.867719,-84.852814,67.082039,-42.426407,42.426407,15187.0,-0.483855
167054,167055.0,6773.0,0.0,831.0,6773.0,-37.634686,496.507613,205.906598,0.0,41151.5,8.0,-1,841.267481,611.882324,1032.279053,611.882324,1032.279053,137.036936,0.0,305.941162,0.0,276.586334,137.036936,0.0,305.941162,0.0,276.586334,1714.179993,1524.106323,1929.818604,1524.106323,1929.818604,38.211187,0.0,152.970581,90.0,152.970581,0.265255,0.187762,0.327465,0.327465,0.187762,0.166083,0.00,0.76,0.76,0.00,0.062072,0.033656,0.090260,0.09026,0.033656,0.145214,0.0,1.0,1.0,0.0,0.145214,0.0,1.0,1.0,0.0,132.056153,-42.426407,305.941162,-42.426407,276.586334,15187.0,-0.208582
167055,167056.0,6774.0,0.0,831.0,6774.0,-18.001250,341.453471,225.540034,0.0,41151.5,8.0,-1,751.950489,611.882324,898.999451,611.882324,898.999451,46.099398,0.0,174.928558,0.0,174.928558,46.099398,0.0,174.928558,0.0,174.928558,1609.762548,1519.078613,1718.138550,1524.106323,1718.138550,28.544933,0.0,94.868332,90.0,30.000000,0.292625,0.251578,0.327465,0.327465,0.251578,0.345966,0.00,0.84,0.76,0.00,0.074239,0.053034,0.090260,0.09026,0.053034,0.470148,0.0,1.0,1.0,0.0,0.470148,0.0,1.0,1.0,0.0,29.996062,-60.000000,174.928558,-42.426407,174.928558,15187.0,-0.291362
167056,167057.0,6775.0,0.0,831.0,6775.0,-144.759947,588.596818,98.781337,0.0,41151.5,8.0,-1,665.477722,600.000000,807.774719,611.882324,807.774719,13.348508,0.0,42.426407,0.0,30.000000,13.348508,0.0,42.426407,0.0,30.000000,1638.924566,1524.106323,1797.998901,1524.106323,1797.998901,47.866174,0.0,108.166542,90.0,0.000000,0.349664,0.327465,0.366976,0.327465,0.356797,0.571184,0.32,0.80,0.76,0.36,0.093646,0.088220,0.100459,0.09026,0.088220,0.556435,0.0,1.0,1.0,0.0,0.556435,0.0,1.0,1.0,0.0,-9.358880,-60.000000,42.426407,-42.426407,30.000000,15187.0,-0.489409


#### Import the Necessary Code

In [4]:
# Import all necessary libraries we will be using in our dataset

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import sklearn
import statistics
import random

from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile, RFE, SelectFromModel

from sklearn.preprocessing import StandardScaler, Binarizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, auc, roc_curve
from IPython.display import display, HTML

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, TimeSeriesSplit, StratifiedShuffleSplit

from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor,AdaBoostClassifier,RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


%matplotlib inline

matplotlib.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')
print(male_clean.shape)


(167057, 69)


### Male clean - Observed vs Random

In [5]:
male_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167057 entries, 0 to 167056
Data columns (total 69 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  167057 non-null  float64
 1   FID         167057 non-null  float64
 2   Id          167057 non-null  float64
 3   Bear_ID     167057 non-null  float64
 4   SAMPLEID    167057 non-null  float64
 5   TURNANGLE   167057 non-null  float64
 6   STEPLENGTH  167057 non-null  float64
 7   BEARING     167057 non-null  float64
 8   OBSERVED    167057 non-null  float64
 9   datetime    167057 non-null  float64
 10  month       167057 non-null  float64
 11  CLUSTER     167057 non-null  object 
 12  disagLWM    167057 non-null  float64
 13  disagMIN    167057 non-null  float64
 14  disagMAX    167057 non-null  float64
 15  disagBEG    167057 non-null  float64
 16  disagEND    167057 non-null  float64
 17  disforLWM   167057 non-null  float64
 18  disforMIN   167057 non-null  float64
 19  di

In [ ]:
# Create plots for the top 4 features as found from lab 2
# cost_per_liter_trans, state_bottle_cost_trans, sale_dollars_trans, totalcost_trans
import seaborn as sns
sns.pairplot(male_clean, height=4, hue= 'OBSERVED', vars = ['TURNANGLE', 'STEPLENGTH', 'BEARING', 'disagLWM', 'disforLWM', 'disnatLWM', 'distrdsLWM', 'distwatLWM', 'natden5LWM', 'fordenLWM', 'rddenLWM', 'forestLWM', 'naturalLWM', 'NatnegLWM'])

KeyboardInterrupt: ignored

# MOdeling and Eval 1

In [ ]:
male_clean.columns

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import metrics as mt

cv = StratifiedKFold(n_splits=10)

features = ['TURNANGLE',
       'STEPLENGTH', 'BEARING', 'datetime', 'month', 
       'disagLWM', 'disagMIN', 'disagMAX', 'disagBEG', 'disagEND', 'disforLWM',
       'disforMIN', 'disforMAX', 'disforBEG', 'disforEND', 'disnatLWM',
       'disnatMIN', 'disnatMAX', 'disnatBEG', 'disnatEND', 'distrdsLWM',
       'distrdsMIN', 'distrdsMAX', 'distrdsBEG', 'distrdsEND', 'distwatLWM',
       'distwatMIN', 'distwatMAX', 'distwatBEG', 'distwatEND', 'natden5LWM',
       'natden5MIN', 'natden5MAX', 'natden5BEG', 'natden5END', 'fordenLWM',
       'fordenMIN', 'fordenMAX', 'fordenBEG', 'fordenEND', 'rddenLWM',
       'rddenMIN', 'rddenMAX', 'rddenBEG', 'rddenEND', 'forestLWM',
       'forestMIN', 'forestMAX', 'forestBEG', 'forestEND', 'naturalLWM',
       'naturalMIN', 'naturalMAX', 'naturalBEG', 'naturalEND', 'NatnegLWM',
       'NatnegMIN', 'NatnegMAX', 'NatnegBEG', 'NatnegEND']

X2 = male_clean[features].copy()

scaler = StandardScaler()
scaler.fit(X2)

#This makes our model's coefficients take on the same scale for accurate feature importance analysis
#Notice we scaled the data before the cross validation
X = scaler.transform(X2)

Y= male_clean[['OBSERVED']].copy()

In [ ]:
from sklearn import metrics as mt

# train and test split before resampling
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y, test_size = 0.2, random_state = 101)

In [ ]:
print("Before OverSampling, counts of label 'Observed': {}".format(sum(y1_train['OBSERVED'] == "1")))
print("Before OverSampling, counts of label 'Random': {} \n".format(sum(y1_train['OBSERVED'] == "0"))) 

# import SMOTE module from imblearn library 
# pip install imblearn (if you don't have imblearn in your system) 
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X1_train_res, y1_train_res = sm.fit_sample(X1_train, y1_train.values.ravel()) 
  
print('After OverSampling, the shape of train_X: {}'.format(X1_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y1_train_res.shape)) 